Sample ID: HCR 3.11 S5  
Section: 200um coronal section from ALM  
Mouse: ANM471122 C57BI/6NCrl, sex M, DOB 10/21/2019  
Genes labelled: Gad1 (514 channel), Ndnf (546 channel), Vip (594 channel), Sst (647 channel)  
Microscope: Zeiss 880 (regular confocal mode)  
Objective: 40X oil immersion 

# Imports

In [75]:
%matplotlib inline    

import napari
from PIL import Image

import numpy as np
import cv2

import matplotlib.pyplot as plt

import time

from os.path import sep
import pickle as pkl
import importlib

from utils import diff_gauss
from utils import find_threshold

from segmentation import get_masks
from segmentation import get_background

from plotting import *

from lipofuscin import annotated_lipo_rois
from lipofuscin import filter_lipo

from expression import *

from classification import *

from cell_depths import *

# Enable autoreload of local modules
%load_ext autoreload
%autoreload 2        # Autoreload all modules except those excluded using %aimport
%aimport napari, PIL.Image, numpy, cv2, matplotlib.pyplot, time, importlib 

# from mpl_toolkits import mplot3d
# import matplotlib.colors as colors
# import matplotlib.path as mpltpath


# import h5py

# from bisect import bisect_left

# from skimage.measure import profile_line
# from skimage.filters import gaussian


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Specify data path

In [4]:
data_path = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.11\\S5'
metadata_file = 'metadata.pkl'

# Metadata

In [41]:
t0 = time.time()

um_per_px = 0.13
raw_image_path = '{0}{1}S5_raw_tiff_images'.format(data_path, sep)
plane_nos = range(1, 11)
base_filename = 'S5__z'
genes = ['Gad1', 'Ndnf', 'Sst', 'Vip']
channel_names = {'Gad1': '_ChS1-T3_ORG',
                 'Ndnf': '_ChS2-T1_ORG',
                 'Vip': '_ChS2-T3_ORG',
                 'Sst': '_ChS2-T2_ORG'}
n_planes = len(plane_nos)
print('Number of planes: {0}'.format(n_planes))

# Create 4D array to store images
img = Image.open('{0}{1}{2}{3}{4}.tif'.format(raw_image_path, sep, base_filename, str(plane_nos[0]).zfill(2),
                                             channel_names[genes[0]]))

h = img.height
w = img.width

cell_data_file = 'cell_data.pkl'
cell_pixels_file = 'cell_pixels.pkl'
all_cell_pixels_file = 'all_cell_pixels.pkl'
bg_pixels_file = 'bg_pixels.pkl'
all_bg_pixels_file = 'all_bg_pixels.pkl'
lipo_rois_file = 'lipo_rois_cells.pkl'
lipo_pixels_cells_file = 'lipo_pixels_cells.pkl'

cell_filt_values_file = 'cell_filt_values.pkl'
bg_filt_values_file = 'bg_filt_values.pkl'
anno_lipo_filt_values_file = 'anno_lipo_filt_values.pkl'

seg_image_path = '{0}{1}S5_blurred_tiff_images'.format(data_path, sep)
plots_path = '{0}{1}Plots'.format(data_path, sep)
filt_image_path = '{0}{1}Filtered_tiff_images'.format(data_path, sep)

metadata = {}
metadata['raw_image_path'] = raw_image_path
metadata['filt_image_path'] = filt_image_path
metadata['plane_nos'] = plane_nos
metadata['base_filename'] = base_filename
metadata['genes'] = genes
metadata['channel_names'] = channel_names
metadata['n_planes'] = n_planes
metadata['h'] = h
metadata['w'] = w
metadata['um_per_px'] = um_per_px

metadata['cell_data_file'] = cell_data_file
metadata['cell_pixels_file'] = cell_pixels_file
metadata['bg_pixels_file'] = bg_pixels_file
metadata['all_bg_pixels_file'] = all_bg_pixels_file
metadata['all_cell_pixels_file'] = all_cell_pixels_file
metadata['lipo_rois_file'] = lipo_rois_file
metadata['lipo_pixels_cells_file'] = lipo_pixels_cells_file

metadata['cell_filt_values_file'] = cell_filt_values_file
metadata['bg_filt_values_file'] = bg_filt_values_file
metadata['anno_lipo_filt_values_file'] = anno_lipo_filt_values_file
    
metadata['seg_image_path'] = seg_image_path
metadata['plots_path'] = plots_path
    
with open('{0}{1}{2}'.format(data_path, sep, metadata_file), 'wb') as f:
    pkl.dump(metadata, f)

Number of planes: 10


# Segmentation

## Get cell pixels

In [27]:
cell_data, n_cells = get_masks.get_masks(data_path, metadata_file)
cell_pixels = get_masks.get_cell_pixels(data_path, metadata_file)
all_cell_pixels = get_masks.get_all_cell_pixels(data_path, metadata_file)

357 cell masks found
Finding pixels in cells
Cell 0: 38.0 seconds
Cell 2: 61.0 seconds
Cell 4: 84.0 seconds
Cell 6: 107.0 seconds
Cell 8: 122.0 seconds
Cell 10: 144.0 seconds
Cell 12: 189.0 seconds
Cell 14: 204.0 seconds
Cell 16: 218.0 seconds
Cell 18: 240.0 seconds
Cell 20: 263.0 seconds
Cell 22: 292.0 seconds
Cell 24: 315.0 seconds
Cell 26: 337.0 seconds
Cell 28: 374.0 seconds
Cell 30: 411.0 seconds
Cell 32: 441.0 seconds
Cell 34: 456.0 seconds
Cell 36: 485.0 seconds
Cell 38: 522.0 seconds
Cell 40: 537.0 seconds
Cell 42: 552.0 seconds
Cell 44: 574.0 seconds
Cell 46: 619.0 seconds
Cell 48: 663.0 seconds
Cell 50: 686.0 seconds
Cell 52: 700.0 seconds
Cell 54: 723.0 seconds
Cell 56: 745.0 seconds
Cell 58: 768.0 seconds
Cell 60: 791.0 seconds
Cell 62: 806.0 seconds
Cell 64: 836.0 seconds
Cell 66: 858.0 seconds
Cell 68: 881.0 seconds
Cell 70: 896.0 seconds
Cell 72: 926.0 seconds
Cell 74: 949.0 seconds
Cell 76: 972.0 seconds
Cell 78: 995.0 seconds
Cell 80: 1026.0 seconds
Cell 82: 1049.0 sec

## Get background pixels

In [93]:
bg_pixels = get_background.get_bg_pixels(data_path, metadata_file)
all_bg_pixels = get_background.get_all_bg_pixels(data_path, metadata_file)

Finding pixels in local background of cells


## Segment cells

# Lipofuscin exclusion

## Annotated lipofuscin rois

In [25]:
l_rois = annotated_lipo_rois.get_rois(data_path, metadata_file)
all_lipo_pixels, lipo_pixels_roi = annotated_lipo_rois.get_cell_lipo_pixels(data_path, metadata_file)

14 rois
Data loaded


## Detect lipofuscin rois in cells and background

In [126]:
# Filter images 
cell_filt_values, bg_filt_values, annotated_lipo_filt_values = filter_lipo.filter(data_path, metadata_file, 
                                                                                   genes = ['Ndnf'],
                                                                                  save_tiffs = True)



Filtering Ndnf images: 0.0 seconds
Plane 1, 0.0 seconds
Saving plane 0: 11.0 seconds
Plane 2, 11.0 seconds
Saving plane 1: 31.0 seconds
Plane 3, 31.0 seconds
Saving plane 2: 58.0 seconds
Plane 4, 58.0 seconds
Saving plane 3: 84.0 seconds
Plane 5, 85.0 seconds


ValueError: Can not convert from object to float64.

# Reload modules

In [125]:
# Ideally, modules should be auto-reloaded 
importlib.reload(get_masks)
importlib.reload(get_background)

importlib.reload(annotated_lipo_rois)
importlib.reload(filter_lipo)

importlib.reload(diff_gauss)
importlib.reload(find_threshold)

<module 'utils.find_threshold' from 'G:\\Shared drives\\as_share\\HCR\\FISH Image Analysis\\utils\\find_threshold.py'>

# Debugging

In [110]:
sigma_small = 5
sigma_large = 10
thresh_scale = 1
save_tiffs = True

import time
import pickle as pkl
from os.path import sep
from PIL import Image
from utils import diff_gauss
from utils import find_threshold
from tifffile import imwrite
from segmentation import get_masks
import numpy as np


# Load metadata
with open('{0}{1}{2}'.format(data_path, sep, metadata_file), 'rb') as f:
    metadata = pkl.load(f)
raw_image_path = metadata['raw_image_path']
filt_image_path = metadata['filt_image_path']
n_planes = metadata['n_planes']
h = metadata['h']
w = metadata['w']
base_filename = metadata['base_filename']
channel_names = metadata['channel_names']
if genes == None:
    genes = metadata['genes']

# Load cell pixels
all_cell_pixels_file = metadata['all_cell_pixels_file']
with open('{0}{1}{2}'.format(data_path, sep, all_cell_pixels_file), 'rb') as f:
    all_cell_pixels = pkl.load(f)

cell_pixels_file = metadata['cell_pixels_file']
with open('{0}{1}{2}'.format(data_path, sep, cell_pixels_file), 'rb') as f:
    cell_pixels = pkl.load(f)

# Load background pixels
bg_pixels_file = metadata['bg_pixels_file']
with open('{0}{1}{2}'.format(data_path, sep, bg_pixels_file), 'rb') as f:
    bg_pixels = pkl.load(f)

# Load annotated lipofuscin pixels
lipo_pixels_cells_file = metadata['lipo_pixels_cells_file']
with open('{0}{1}{2}'.format(data_path, sep, lipo_pixels_cells_file), 'rb') as f:
    data_dict = pkl.load(f)
all_lipo_pixels = data_dict['all_lipo_pixels']

# Allocate variables to store pixel values in filtered images
cell_filt_values = {cell: {} for cell in cell_pixels.keys()}
bg_filt_values = {cell: {} for cell in cell_pixels.keys()}
annotated_lipo_filt_values = []

t0 = time.time()
im_thresh = np.zeros([h, w])
for gene in genes:

    print('Filtering {0} images: {1} seconds'.format(gene, np.round(time.time() - t0)))

    for plane in range(n_planes):
        print('Plane {0}, {1} seconds'.format(plane + 1, np.round(time.time() - t0)))
        img = Image.open('{0}{1}{2}{3}{4}.tif'.format(raw_image_path, sep, base_filename, str(plane + 1).zfill(2),
                                                        channel_names[gene]))
        im_array = np.array(img)
        im_diff = diff_gauss.diff_gauss(sigma_small, sigma_large, im_array)

        px = all_cell_pixels[all_cell_pixels[:, 0] == plane, 1:]
        points = im_diff[px[:, 0], px[:, 1]]
        thresh = find_threshold.find_threshold(points, thresh_scale)

        ix = np.where(im_diff > thresh)
        im_thresh[ix] = im_diff[ix]


        if save_tiffs:
            print('Saving plane {0}: {1} seconds'.format(plane, np.round(time.time() - t0)))
            imwrite('{0}{1}{2}_plane{3}_filt_sigma={4},{5}_thresh={6}.tif'.format(filt_image_path, sep, gene,
                                                                            str(plane + 1).zfill(2),
                                                                            sigma_small, sigma_large,
                                                                            np.round(thresh)),
                                                                            im_thresh.astype(np.int16))

        # Get pixel values for cells and background in filtered images
        cells_in_plane = get_masks.get_cells_in_plane(data_path, metadata_file, plane)
        for cell in cells_in_plane:

            cell_px = cell_pixels[cell][plane]
            cell_filt_values[cell][plane] = im_thresh[cell_px[0], cell_px[1]]

            bg_px = bg_pixels[cell][plane]
            bg_filt_values[cell][plane] = im_thresh[bg_px[:, 0], bg_px[:, 1]]

        # Get pixel values for annotated lipofuscin pixels in filtered images
        lipo_px = all_lipo_pixels[all_lipo_pixels[:, 0] == plane, 1:]
        annotated_lipo_filt_values = np.append(annotated_lipo_filt_values, im_thresh[lipo_px[:, 0], lipo_px[:, 0]])



Filtering Gad1 images: 0.0 seconds
Plane 1, 0.0 seconds


NameError: name 'save_tiffs' is not defined